<a href="https://colab.research.google.com/github/passionyang16/context_summarize/blob/main/%E1%84%80%E1%85%A9%E1%84%82%E1%85%A1%E1%86%AB%E1%84%80%E1%85%AA%E1%84%8B%E1%85%A7%E1%86%A8%E1%84%80%E1%85%A7%E1%86%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

     |████████████████████████████████| 133.6MB 49kB/s 
     |████████████████████████████████| 61kB 3.5MB/s 
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12


In [ ]:
import torch_xla
import torch_xla.core.xla_model as xm

In [ ]:
device = xm.xla_device()

## 구글 마운트

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Mounted at /content/gdrive


In [ ]:
cd gdrive/My Drive/

/content/gdrive/My Drive


In [ ]:
import os
from pathlib import Path

folder = "context_summarize" ## 자기 드라이브 경로 입력

base_path = Path("세줄이/")
project_path = base_path / folder
os.chdir(project_path)
for x in list(project_path.glob("*")):
    if x.is_dir():
        dir_name = str(x.relative_to(project_path))
        os.rename(dir_name, dir_name.split(" ", 1)[0])
print(f"{os.getcwd()}")

/content/gdrive/My Drive/세줄이/context_summarize


## KoBERT 및 기타 라이브러리 설치

In [ ]:
!git clone https://github.com/SKTBrain/KoBERT.git

fatal: destination path 'KoBERT' already exists and is not an empty directory.


In [ ]:
cd KoBERT

/content/gdrive/My Drive/세줄이/context_summarize/KoBERT


In [ ]:
ls

imgs/  kobert/  LICENSE  logs/  README.md  requirements.txt  scripts/  setup.py


In [ ]:
!pip install .

Processing /content/gdrive/My Drive/세줄이/context_summarize/KoBERT
  Created wheel for kobert: filename=kobert-0.1.2-cp36-none-any.whl size=12734 sha256=04d00fefcb89a22e4b802f75e04b2bc4238e48892c08ee61306c27f64e18778c
  Stored in directory: /tmp/pip-ephem-wheel-cache-d0csrol7/wheels/6b/63/d5/70ce1c6dd21f92c6e746b9ea63798e7c0e3dce4148f2adb7f0
Successfully built kobert


In [ ]:
!pip install -r requirements.txt
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install transformers==2.1.1
!pip install torch==1.3.1

     |████████████████████████████████| 55.0MB 2.1MB/s 
     |████████████████████████████████| 348kB 60.1MB/s 
     |████████████████████████████████| 1.1MB 54.2MB/s 
     |████████████████████████████████| 4.1MB 42.4MB/s 
     |████████████████████████████████| 1.5MB 61.4MB/s 
     |████████████████████████████████| 890kB 63.4MB/s 
     |████████████████████████████████| 2.9MB 61.6MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588516 sha256=07524bd2d7894133975ad895044b1aff6846cdad638f5b6c4b2d6e66e3863251
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=f6de69133172a6ccc1124f43d60103e501fb143e0f0bdaa6f7c428ad90f419bc
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built gluonnlp sacremoses
  Found existing installati

In [ ]:
cd KoBERT/

[Errno 2] No such file or directory: 'KoBERT/'
/content/gdrive/My Drive/세줄이/context_summarize/KoBERT


## 패키지 로드

In [ ]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm, tqdm_notebook

# pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

#Sentence Tokenizer
from gluonnlp.data import SentencepieceTokenizer


# KoBERT
from kobert.pytorch_kobert import get_pytorch_kobert_model
from kobert.utils import get_tokenizer

# etc
import gluonnlp as nlp
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule

In [ ]:
cd ..

/content/gdrive/My Drive/세줄이/context_summarize


## Colab GPU 설정

In [ ]:
#device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

xla:1


## KoBERT Tokenizer 생성

In [ ]:
# KoBERT pretrained model 다운로드 후 모델 가져오기
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
# KoBERT pretrained tokenizer 다운로드 후 tokenizer 생성
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
# 특수 토큰
for k in range(10):
  print(tok.vocab.idx_to_token[k])

[UNK]
[PAD]
[CLS]
[SEP]
[MASK]
!
!'
!”
"
#


 [UNK] : unknown token

 [PAD] : The token used for padding, for example when batching sequences of different lengths.

 [CLS] : The classifier token which is used when doing sequence classification (classification of the whole sequence instead of per-token classification).
         It is the first token of the sequence when built with special tokens.

 [SEP] : The separator token, which is used when building a sequence from multiple sequences, e.g. two sequences for sequence classification or for a text
         and a question for question answering. It is also used as the last token of a sequence built with special tokens.
         
 [MASK]: The token used for masking values. This is the token used when training this model with masked language modeling.
         This is the token which the model will try to predict.

## 하이퍼파라미터 설정

In [ ]:
max_sentence_num = 64
max_word_num = 64
num_workers = 5
batch_size = 64
learning_rate = 1e-5

## Dataset 클래스

In [ ]:
from tqdm import tqdm

class ArticleDataset(Dataset):
  """
  기사 데이터 로드

  input :
      dataframe : 기사 데이터
      article_col : 
  """

  def __init__(self, dataframe, article_col, ext_col, tokenizer, max_sentence_num=50, max_word_num=100, pad=True, pair=False):

    # data
    self.articles = []

    transform = nlp.data.BERTSentenceTransform(tokenizer, max_seq_length=max_word_num, pad=pad, pair=pair)
    total_words_tokenized = []
    
    for article in tqdm(dataframe[article_col]):
      size = max_sentence_num * max_word_num
      numbers_tokenized = np.array([])

      for sentence in article:
        # tokenizing
        tokenized_to_num = transform([sentence])
        numbers_tokenized = np.concatenate((numbers_tokenized, tokenized_to_num[0]), axis = 0)
      
      # padding
      if len(numbers_tokenized) < size:
        remaining_adds = size - len(numbers_tokenized)
        zeros_to_add = np.ones((remaining_adds,),dtype=np.int32)
        numbers_tokenized = np.concatenate((numbers_tokenized,zeros_to_add),axis=0)
      else:
        numbers_tokenized = numbers_tokenized[:size]
      
      numbers_tokenized = numbers_tokenized.astype(np.int32)
      self.articles.append(numbers_tokenized)

    # label
    self.summary = []
    for extractive in dataframe[ext_col]:
      binary = self.binary_target_processor(extractive)
      self.summary.append(binary)

  def binary_target_processor(self, extractive):
    binary = np.zeros((max_sentence_num,1),dtype=np.int32)
    for sentence_index in extractive:
      if sentence_index < max_sentence_num:
        binary[sentence_index] = 1
    return binary

  def __getitem__(self, i):
    return (self.articles[i], self.summary[i])
  
  def __len__(self):
    return len(self.articles)

In [ ]:
from tqdm import tqdm

class ArticleLineDataset(Dataset):
  """
  기사 데이터 로드

  input :
      dataframe : 기사 데이터
      article_col : 
  """

  def __init__(self, dataframe, article_col, ext_col, tokenizer, max_sentence_num=128, max_word_num=128, pad=True, pair=False):
    transform = nlp.data.BERTSentenceTransform(tokenizer, max_seq_length=max_word_num, pad=pad, pair=pair)    

    # data
    self.articles = []
    self.sentence_length = []
    for article in tqdm(dataframe[article_col]):
      numbers_tokenized = np.array([[]])

      for sentence in article:
        # tokenizing
        tokenized_to_num = transform([sentence])
        tokenized_to_num = np.expand_dims(tokenized_to_num[0], axis=0)
        if numbers_tokenized.shape == (1, 0):
          numbers_tokenized = np.concatenate((tokenized_to_num, ), axis = 0)
        else:
          numbers_tokenized = np.concatenate((numbers_tokenized, tokenized_to_num), axis = 0)

      self.sentence_length.append(np.int32(len(numbers_tokenized)))

      # padding
      if len(numbers_tokenized) < max_sentence_num:
        remaining_adds = max_sentence_num - len(numbers_tokenized)
        padding_sentence = np.ones((remaining_adds, max_word_num),dtype=np.int32)
        numbers_tokenized = np.concatenate((numbers_tokenized, padding_sentence),axis=0)
      else:
        numbers_tokenized = numbers_tokenized[:max_sentence_num]
      
      numbers_tokenized = numbers_tokenized.astype(np.int32)
      self.articles.append(numbers_tokenized)

    # label
    self.summary = []
    for extractive in dataframe[ext_col]:
      binary = self.binary_target_processor(extractive)
      self.summary.append(binary)

  def binary_target_processor(self, extractive):
    binary = np.zeros((max_sentence_num,1),dtype=np.int32)
    for sentence_index in extractive:
      if sentence_index < max_sentence_num:
        binary[sentence_index] = 1
    return binary

  def __getitem__(self, i):
    return (self.articles[i], self.sentence_length[i], self.summary[i])
  
  def __len__(self):
    return len(self.articles)

## 데이터 불러오기

In [ ]:
!ls

KoBERT


In [ ]:
cd ..

/content/gdrive/My Drive/세줄이


In [ ]:
df = pd.read_json("data/train.jsonl", lines=True)

In [ ]:
# train_dataset = ArticleDataset(df, "article_original", "extractive", tok, max_sentence_num, max_word_num, True, False)
train_dataset = ArticleLineDataset(df, "article_original", "extractive", tok, max_sentence_num, max_word_num, True, False)

100%|██████████| 42803/42803 [01:46<00:00, 400.51it/s]


In [ ]:
class MyDataLoader():
  def __init__(self, dataset, batch_size):
    self.current = 0
    self.dataset = [item for item in dataset]
    self.batch_size = batch_size
    if len(dataset) % batch_size == 0:
      self.stop = len(dataset) // batch_size
    else: 
      self.stop = len(dataset) // batch_size + 1
    self.batch = self.devide_batch()

  def devide_batch(self):
    import random

    random.shuffle(self.dataset)

    batchs = [ [] for _ in range(self.stop) ]

    for i, dt in enumerate(self.dataset):
      batchs[i % self.stop].append((torch.from_numpy(dt[0]).unsqueeze(0), torch.tensor(dt[1]).unsqueeze(0), torch.from_numpy(dt[2]).unsqueeze(0)))
    
    return batchs

  def __iter__(self):
    return self

  def __next__(self):
    if self.current < self.stop:    # 현재 숫자가 반복을 끝낼 숫자보다 작을 때
      flag = True
      for article, num, ext in self.batch[self.current]:
        if flag:
          articles = torch.cat([article, ], dim=0)
          nums = torch.cat([num, ], dim=0)
          exts = torch.cat([ext, ], dim=0)
          flag = False
        else:
          articles = torch.cat([articles, article], dim=0)
          nums = torch.cat([nums, num], dim=0)
          exts = torch.cat([exts, ext], dim=0)

      self.current += 1           # 현재 숫자를 1 증가시킴
      return [articles, nums, exts]
    else:                           # 현재 숫자가 반복을 끝낼 숫자보다 크거나 같을 때
      self.current = 0
      random.shuffle(self.dataset)
      raise StopIteration


In [ ]:
loader = MyDataLoader(train_dataset, batch_size=batch_size)

## 모델

In [ ]:
#positional encoding function
def get_sinusoid_encoding_table(n_seq, d_hidn):
    def cal_angle(position, i_hidn):
        return position / np.power(10000, 2 * (i_hidn // 2) / d_hidn)
    def get_posi_angle_vec(position):
        return [cal_angle(position, i_hidn) for i_hidn in range(d_hidn)]

    sinusoid_table = np.array([get_posi_angle_vec(i_seq) for i_seq in range(n_seq)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # even index sin 
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # odd index cos

    return sinusoid_table

In [ ]:
#Mask padding function
def get_attn_pad_mask(seq_q, seq_k, i_pad):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    pad_attn_mask = seq_k.data.eq(i_pad).unsqueeze(1).expand(batch_size, len_q, len_k)
    return pad_attn_mask

In [ ]:
# ScaledDotProductAttention Class
class ScaledDotProductAttention(nn.Module):
  def __init__(self, d_head):
    super().__init__()
    self.scale = 1 / (d_head **0.5)
  def forward(self, Q, K, V, attn_mask):
    scores = torch.matmul(Q, K.transpose(-1,-2)).mul_(self.scale)
    scores.masked_fill_(attn_mask, -1e9)
    attn_prob = nn.Softmax(dim=-1)(scores)
    context = torch.matmul(attn_prob, V)
    return context, attn_prob

In [ ]:
# MultiHeadAttention Class
class MultiHeadAttention(nn.Module):
    def __init__(self, d_hidn, n_head, d_head):
        super().__init__()
        self.d_hidn = d_hidn
        self.n_head = n_head
        self.d_head = d_head

        self.W_Q = nn.Linear(d_hidn, n_head * d_head)
        self.W_K = nn.Linear(d_hidn, n_head * d_head)
        self.W_V = nn.Linear(d_hidn, n_head * d_head)
        self.scaled_dot_attn = ScaledDotProductAttention(d_head)
        self.linear = nn.Linear(n_head * d_head, d_hidn)
    
    def forward(self, Q, K, V, attn_mask):
        batch_size = Q.size(0)
        q_s = self.W_Q(Q).view(batch_size, -1, self.n_head, self.d_head).transpose(1,2)
        k_s = self.W_K(K).view(batch_size, -1, self.n_head, self.d_head).transpose(1,2)
        v_s = self.W_V(V).view(batch_size, -1, self.n_head, self.d_head).transpose(1,2)

        attn_mask = attn_mask.unsqueeze(1).repeat(1, self.n_head, 1, 1)

        context, attn_prob = self.scaled_dot_attn(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.n_head * self.d_head)
        output = self.linear(context)

        return output, attn_prob

In [ ]:
#PoswiseFeedForwardNet Class
class PoswiseFeedForwardNet(nn.Module):
  def __init__(self, d_hidn):
    super().__init__()
    self.d_hidn = d_hidn

    self.conv1 = nn.Conv1d(in_channels=d_hidn, out_channels=d_hidn*4, kernel_size=1)
    self.conv2 = nn.Conv1d(in_channels=d_hidn*4, out_channels=d_hidn, kernel_size=1)
    self.active = F.gelu

  def forward(self, inputs):
    output = self.active(self.conv1(inputs.transpose(1,2)))
    output = self.conv2(output).transpose(1,2)
    return output

In [ ]:
""" encoder layer """
class EncoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.self_attn = MultiHeadAttention(self.config.d_hidn, self.config.n_head, self.config.d_head)
        self.layer_norm1 = nn.LayerNorm(self.config.d_hidn, eps=self.config.layer_norm_epsilon)
        self.pos_ffn = PoswiseFeedForwardNet(self.config.d_hidn)
        self.layer_norm2 = nn.LayerNorm(self.config.d_hidn, eps=self.config.layer_norm_epsilon)
    
    def forward(self, inputs, attn_mask):
        att_outputs, attn_prob = self.self_attn(inputs, inputs, inputs, attn_mask)
        att_outputs = self.layer_norm1(inputs + att_outputs)
        ffn_outputs = self.pos_ffn(att_outputs)
        ffn_outputs = self.layer_norm2(ffn_outputs + att_outputs)
        return ffn_outputs, attn_prob

In [ ]:

""" encoder """
class Encoder(nn.Module):
    def __init__(self, config, n_layer):
        super().__init__()
        self.config = config
        self.enc_emb = nn.Embedding(self.config.n_enc_vocab, self.config.d_hidn)
        sinusoid_table = torch.FloatTensor(get_sinusoid_encoding_table(self.config.n_enc_seq + 1, self.config.d_hidn))
        self.pos_emb = nn.Embedding.from_pretrained(sinusoid_table, freeze=True)
        self.n_layer = n_layer
        self.layers = nn.ModuleList([EncoderLayer(self.config) for _ in range(self.n_layer)])
    
    def forward(self, inputs):
        positions = torch.arange(inputs.size(1), device=inputs.device, dtype=inputs.dtype).expand(inputs.size(0), inputs.size(1)).contiguous() + 1
        pos_mask = inputs.eq(self.config.i_pad)
        positions.masked_fill_(pos_mask, 0)

        outputs = self.enc_emb(inputs) + self.pos_emb(positions)
        attn_mask = get_attn_pad_mask(inputs, inputs, self.config.i_pad)

        attn_probs = []
        for layer in self.layers:
            outputs, attn_prob = layer(outputs, attn_mask)
            attn_probs.append(attn_prob)
        return outputs, attn_probs

In [ ]:
""" encoder """
class SecondEncoder(Encoder):
    def __init__(self, config, n_layer):
        super().__init__(config, n_layer)

    
    def forward(self, inputs, cls_inputs):
        positions = torch.arange(cls_inputs.size(1), device=device, dtype=cls_inputs.dtype).expand(cls_inputs.size(0), cls_inputs.size(1)).contiguous() + 1
        pos_mask = cls_inputs.eq(self.config.i_pad)
        positions.masked_fill_(pos_mask, 0)

        outputs = inputs + self.pos_emb(positions)
        attn_mask = get_attn_pad_mask(cls_inputs, cls_inputs, self.config.i_pad)

        attn_probs = []
        for layer in self.layers:
            outputs, attn_prob = layer(outputs, attn_mask)
            attn_probs.append(attn_prob)
        return outputs, attn_probs

In [ ]:
class BertEncoder(nn.Module):
  def __init__(self, bert):
    super(BertEncoder, self).__init__()
    self.bert = bert

  def gen_attention_mask(self, token_ids):
    ## masked attenion, 패딩에 패널티 부여, 학습 x
    attention_mask = token_ids.ne(1)
    return attention_mask.float()
  
  def forward(self, token_ids):
    attention_mask = self.gen_attention_mask(token_ids)
    segment_ids = torch.zeros(token_ids.size())

    output, hidden = self.bert(input_ids = token_ids.long(), token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))

    return output, hidden

In [ ]:
class DimensionReducer(nn.Module):
  def __init__(self, input, output):
    super(DimensionReducer, self).__init__()
    self.input = input
    self.output = output
    self.fc1 = nn.Linear(self.input,(self.input + self.output)//2)
    self.fc2 = nn.Linear((self.input + self.output)//2,self.output)

  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))

    return x

In [ ]:
class BinaryClassifier(nn.Module):
  def __init__(self, input):
    super(BinaryClassifier, self).__init__()

    self.input = input
    self.fc1 = nn.Linear(self.input,128)
    self.fc2 = nn.Linear(128,64)
    self.fc3 = nn.Linear(64,32)
    self.fc4 = nn.Linear(32,1)
    
  def forward(self,x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.relu(self.fc3(x))
    x = torch.sigmoid(self.fc4(x))
    return x


In [ ]:
class BERTSummarizer(nn.Module):
  def __init__(self, config, reducer, transformer, classifier, device):
    super().__init__()
    self.config = config
    self.reducer = reducer
    self.second_encoder = transformer
    self.classifier = classifier
    self.device = device

  def forward(self, new_batch, num):

    # reduce dimension for ram space
    new_batch = self.reducer(new_batch)

    # second layer  >> [batch, sentence_num, vector]
    cls_mask = torch.tensor([[1 if i < num[b] else 0 for i in range(new_batch[b].size()[0])] for b in range(len(new_batch))])
    cls_mask = cls_mask.to(device)
    new_batch, attn_prob = self.second_encoder(new_batch, cls_mask)

    # classifier  >> [batch, sentence_num, 1]
    new_batch = self.classifier(new_batch)

    return new_batch

In [ ]:
def embedding(encoder, batch, num):
  flag = True
  for i in range(len(batch)):
    output, hidden = encoder(batch[i][:num[i]].long())
    pad_num = max_sentence_num - num[i]
    if pad_num < 0:
      pad_num = 0
    article = torch.cat([output[:, 0, :], torch.ones(pad_num,768)], dim=0).unsqueeze(0)
    if flag:
      embedded_batch = torch.cat([article, ], dim=0)
      flag=False
    else:
      embedded_batch = torch.cat([embedded_batch, article], dim=0)
  return embedded_batch


# 모델 초기화

In [ ]:
#Config
class Config(dict):
  __getattr__ = dict.__getitem__
  __setattr__ = dict.__setitem__

  @classmethod
  def load(cls, file):
    with open(file,'r') as f:
      config = json.loads(f.read())
      return Config(config)

config = Config({
    "n_enc_vocab": len(vocab),
    "n_dec_vocab": len(vocab),
    "n_enc_seq":128,
    "n_layer":6,
    "d_hidn":128,
    "i_pad":1,
    "d_ff":1024,
    "n_head":4,
    "d_head":64,
    "dropout":0.1,
    "layer_norm_epsilon":1e-12
})

In [ ]:
# bert 기존 모델만 프리징
for name, param in bertmodel.named_parameters():
  param.requires_grad = False

In [ ]:
encoder = BertEncoder(bertmodel)
# encoder = encoder.to(device)
reducer = DimensionReducer(768, 128)
second = SecondEncoder(config=config, n_layer=1)
classifier = BinaryClassifier(128)

model = BERTSummarizer(config, reducer, second, classifier, device)
model = model.to(device) 

# 학습

In [ ]:
learning_rate = 0.01
num_epochs = 700

# Model , Optimizer, Loss

optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
loss_fn = nn.BCELoss()

In [ ]:
def save_checkpoint(epoch, model, optimizer, PATH):
  torch.save({
    'epoch': epoch,
    'model_state_dict': model.to('cpu').state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    }, PATH)

In [ ]:
#forward loop
losses = []
for epoch in range(0, num_epochs):

  model.train()
  print(epoch)
  for batch, num, label in loader:
    # bert embedding
    new_batch = embedding(encoder, batch, num)

    
    new_batch = new_batch.to(device)
    label = label.to(device)

    #calculate output
    output = model(new_batch, num)

    #calculate loss
    loss = loss_fn(output.reshape(-1,1), label.float().reshape(-1,1))
    print(loss)


    #backprop
    optimizer.zero_grad()
    loss.backward()
    xm.optimizer_step(optimizer, barrier=True) #TPU를 쓰기 위해 필요한 코드
    # optimizer.step()

  if epoch%50 == 0:
    print("epoch {}\tloss : {}".format(epoch,loss))
   
  #  ## checkpoint 저장
  # if (epoch+1) % 10 == 0:
  if not os.path.exists("checkpoints"):
      os.makedirs("checkpoints")
  save_checkpoint(epoch+1, model, optimizer, f"checkpoints/checkpoint_{str(epoch+1)}.pt")
  print("checkpoint saved!")

0
tensor(0.6045, device='xla:1', grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6014, device='xla:1', grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5986, device='xla:1', grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5954, device='xla:1', grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5923, device='xla:1', grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5894, device='xla:1', grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5865, device='xla:1', grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5834, device='xla:1', grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5809, device='xla:1', grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5778, device='xla:1', grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5744, device='xla:1', grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5717, device='xla:1', grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5688, device='xla:1', grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5663, device='xla:1', grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5630, device='xla:1', g

RuntimeError: ignored

# 모델 불러오기

In [ ]:
# 모델 불러오기
save_path = '/content/gdrive/My Drive/context_summarize/checkpoints/checkpoint_1.tar'
checkpoint = torch.load(save_path)
model.load_state_dict(checkpoint['model_state_dict'])

# input을 이제 dataloader에 넣어야하는데 이부분에서 df[:1] 대신 사용자의 데이터를 넣어야할 것 같은데,,, 음.. 이게 가능하려나 어렵네,
# 내가 생각하기에 input을 하나만 넣으면 맞춰서 돌아가는 구조? --> 정 안되면 로컬 dataloader나 saver를 사용하는 것도 좋을 듯한데
train_dataset = ArticleLineDataset(df[:1], "article_original", "extractive", tok, max_sentence_num, max_word_num, True, False)
loader = MyDataLoader(train_dataset, batch_size=batch_size)

# 모델 불러서 output
for batch, num, label in loader:
    # bert embedding
    new_batch = embedding(encoder, batch, num)

    new_batch = new_batch.to(device)
    label = label.to(device)
    #calculate output
    output = model(new_batch, num)

print(output.shape)
# outputsize가 
test = output[0].squeeze(1)
print(test.shape)

result = []
for i, out in enumerate(test):
  result.append([i,out])

result.sort(key=lambda x: x[1], reverse=True)

top_three_sentences = result[0:3]
top_three_sentences

In [ ]:
# 모델 불러오기
save_path = '/content/gdrive/My Drive/context_summarize/'
checkpoint = torch.load(save_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scheduler.load_state_dict(checkpoint['scheduler_state_dict'])